In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
import string
import json
import pickle
from collections import defaultdict

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
from urllib2 import urlopen

In [9]:
test1 = open("anime_pages.p",'rb') 
anime = pickle.load(test1)  
test1.close()

In [11]:
root = 'http://www.anime-planet.com'

all_anime = []
for page in anime:
    soup = BeautifulSoup(page.text, "html.parser")
    animelist = soup.find('ul',attrs={"class": "cardDeck pure-g cd-narrow"}).find_all("li")
    for ani in animelist:
        all_anime.append(root + ani.find('a').get('href'))

In [112]:
%%time
anime_reviews1 = defaultdict(list)
for i in xrange(len(all_anime)):
    review = requests.get(all_anime[i]+"/reviews")
    soup1 = BeautifulSoup(review.text, "html.parser")
    pages = soup1.find('section',attrs={'pure-g'}).find('div',attrs='pure-u-1 pure-u-md-4-5')
    if pages is not None:
        pages = pages.find('div',attrs='pagination aligncenter')
        if pages is not None:
            all_li = pages.ul.find_all('li')
            maxpage = int(all_li[len(all_li)-2].a.contents[0])
        else:
            maxpage=1
    else:
        maxpage=1

    for j in xrange(1,maxpage+1):
        anime_reviews1[all_anime[i].split('/')[-1]].append(requests.get(all_anime[i]+"/reviews?page="+str(j)))
        time.sleep(1)

CPU times: user 3min 32s, sys: 21.2 s, total: 3min 54s
Wall time: 2h 44min 30s


In [2]:
#pickle.dump(anime_reviews, open("anime_reviews.p", "wb"))

test3 = open("anime_reviews.p",'rb') 
anime_reviews = pickle.load(test3)  
test3.close()

In [ ]:
#anime_reviews['digimon-adventure-tri-movie-2'][0].text

In [127]:
reviewsdict = {}
for anime in anime_reviews.keys():
    animedict = {anime:{}}
    for i in range(len(anime_reviews[anime])):
        soup = BeautifulSoup(anime_reviews[anime][i].text, "html.parser")
        body = soup.find('div',attrs={"class": "pure-u-1 pure-u-md-4-5"})
        if body is not None:
            sections = body.find_all('section', attrs={'class':'pure-g'})
            names = [i.find('a',attrs={'itemprop':'author'}).contents[0] for i in sections if i.find('a',attrs={'itemprop':'author'}) is not None]
            scores = [i.find_all('div',attrs={'class':'pure-u-1-5'}) for i in sections if i.find_all('div',attrs={'class':'pure-u-1-5'}) != []]
            story = [i[0].contents[0] for i in scores]
            animation = [i[1].contents[0] for i in scores]
            sound = [i[2].contents[0] for i in scores]
            characters = [i[3].contents[0] for i in scores]
            overall = [i[4].contents[0] for i in scores]
            reviews = [i.contents[1].string for i in body.find_all('section', attrs={'class':'userContent readMore'})]
            temp = dict(zip(names,zip(story,animation,sound,characters,overall,reviews)))
            animedict[anime].update(temp)
    reviewsdict.update(animedict)

In [95]:
# with open('all_reviews.json', 'w') as fp:
#       json.dump(reviewsdict, fp)

with open('../data/all_reviews.json','r') as data_file:    
    reviewsdict = json.load(data_file)

In [96]:
len(reviewsdict)

7282

In [97]:
reviewlist = []
for i in reviewsdict.keys():
    reviewlist+=reviewsdict[i].keys()
users = list(set(reviewlist))

In [98]:
len(users),len(reviewlist)

(4420, 17019)

In [109]:
newlist = []

for key in reviewsdict:
    for user in reviewsdict[key]:
        story = reviewsdict[key][user][0]
        animation = reviewsdict[key][user][1]
        sound = reviewsdict[key][user][2]
        character = reviewsdict[key][user][3]
        overall = reviewsdict[key][user][4]
        reviews = reviewsdict[key][user][5]
        temp = {"name":user,'anime':key,'story':story,'animation':animation,'sound':sound,'character':character,'overall':overall,'reviews':reviews}
        newlist.append(temp)

In [1]:
newdata

NameError: name 'newdata' is not defined

In [111]:
newdata = pd.DataFrame(newlist)
newdata.head()

,animation,anime,character,name,overall,reviews,sound,story
0,\n5/10,dokkoida,\n6/10,angelsreviews,\n5/10,None,\n5/10,\n8/10
1,\n9/10,inazuma-eleven,\n9/10,xxjacobxx3,\n9/10,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,\n9/10,\n9/10
2,\n8/10,inazuma-eleven,\n6/10,Lumineux,\n7/10,Story: 6,\n8/10,\n6/10
3,\n6.5/10,wizardry,\n5/10,sothis,\n7/10,None,\n6/10,\n8/10
4,\n7/10,wizardry,\n8/10,Pauwolf,\n7/10,"Inspired by an old videogame, this 50 minutes ...",\n7/10,\n6/10


In [ ]:
newdata.to_csv('../data/all_reviews.csv',sep='\t',encoding='utf-8')

#### Getting User Profile

In [8]:
%%time
root = 'http://www.anime-planet.com/users/'
user_profiles = {}
for i in xrange(len(users)):
    user = requests.get(root+users[i])
    soup = BeautifulSoup(user.text, "html.parser")
    user_profiles[users[i]]=soup
    time.sleep(1)

CPU times: user 3min 17s, sys: 26.8 s, total: 3min 44s
Wall time: 1h 46min 45s


In [91]:
each_user_info = {}

for user in user_profiles.keys():
    test = user_profiles[user]
    user_info = {}

    info = test.find('div',attrs={'class':'pure-u-3-5 pure-u-md-4-5'})
    name = info.find('h2',attrs={'id':'profileName'}).find_all('a')[1].contents[0]

    stats = info.find('ul',attrs={'class':'userStats'}).findAll('li')
    place = ""
    age_sex = ""
    join = ""

    # find place, age, sex if exist
    for li in stats:
        if li.findAll(attrs={"class" : "fa fa-home"}):
            place = li.contents[1]
        elif li.findAll(attrs={"class" : "fa fa-calendar"}):
            join = li.contents[1]
        elif li.findAll(attrs={"class" : "fa fa-user"}):
            age_sex = li.contents[2].replace('\n','')

    each_user_info[name] = {"place":place,"age_sex":age_sex,"join":join}

In [2]:
# with open('../data/all_users.json', 'w') as fp:
#        json.dump(each_user_info, fp)

with open('../data/all_users.json','r') as data_file:    
    each_user_info = json.load(data_file)

IOError: [Errno 2] No such file or directory: '../data/all_users.json'

In [115]:
userdf = pd.DataFrame(each_user_info).T
userdf.head()

,age_sex,join,place
00081,26,"Joined Jan 16, 2012",Death Star
0Marcandre,20 / M,"Joined Mar 31, 2013",
0oDaano0,?,"Joined Apr 9, 2013",
0utl4w,?,"Joined Mar 31, 2010",somewhere
11bowman,21 / M,"Joined Jun 19, 2010","Amsterdam, Netherlands"


In [ ]:
userdf.to_csv('../data/all_users.csv',sep='\t',encoding='utf-8')